# Step 2: Execute the AI Meeting Recorder Agent with Memory

In this notebook, we'll demonstrate how to interact with the AI Meeting Recorder Agent we configured in Step 1. We'll focus on:

1. Loading the agent from our saved configuration
2. Running the agent with memory persistence
3. Retrieving calendar events (with Google Calendar authorization)
4. Analyzing agent responses
5. Example use cases for the meeting recorder

## Prerequisites

- You've completed Step 1 (agent configuration)
- Your `.env` file contains the necessary API keys
- You have access to the Google Calendar API

## 1. Setup and Environment Configuration

First, let's import the required libraries and set up our agent runner.

In [2]:
import os
import datetime
import time
from dotenv import load_dotenv
from xpander_sdk import XpanderClient, LLMProvider, ToolCall, Tokens, LLMTokens, Agent
from openai import OpenAI

# Load environment variables
load_dotenv()

# Verify required API keys are available
xpander_api_key = os.environ.get("XPANDER_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")
agent_id = os.environ.get("XPANDER_AGENT_ID")

if not all([xpander_api_key, openai_api_key, agent_id]):
    missing = []
    if not xpander_api_key: missing.append("XPANDER_API_KEY")
    if not openai_api_key: missing.append("OPENAI_API_KEY")
    if not agent_id: missing.append("XPANDER_AGENT_ID")
    print(f"❗️ Missing required environment variables: {', '.join(missing)}")
else:
    print(f"✅ All required API keys loaded successfully")
    
print(f"Open the agent alogside this notebook at https://app.xpander.ai/agents/{agent_id}")

✅ All required API keys loaded successfully
Open the agent alogside this notebook at https://app.xpander.ai/agents/6bb96105-3675-4d79-9dd3-8ae146cff007


## 2. Define the Meeting Agent Class

Let's implement our `MeetingAgent` class that will handle interactions with the agent:

In [4]:
class MeetingAgent:
    """Class for running the meeting recorder agent"""
    
    def __init__(self, openai_api_key, xpander_api_key, agent_id):
        self.agent_id = agent_id
        self.xpander_client = XpanderClient(api_key=xpander_api_key)
        self.openai_client = OpenAI(api_key=openai_api_key)
        self.agent : Agent = self.xpander_client.agents.get(agent_id=self.agent_id)

    def run(self, prompt=None, thread_id=None):
        """Run the agent with the given prompt"""
        
        task = prompt or f"Please check the status of all recorded meetings."

        print("-" * 60)
        print(f"{'Continuing conversation in thread: ' + thread_id if thread_id else 'Starting a new conversation thread'}")
        print(f"Processing Task: \n\n{task}\n\n")
        print("-" * 60)
                
        # Create task with or without thread_id
        self.agent.add_task(input=task, thread_id=thread_id if thread_id else None)
                # Initialize token tracking and timing
        execution_tokens = Tokens(worker=LLMTokens(completion_tokens=0, prompt_tokens=0, total_tokens=0))
        execution_start_time = time.perf_counter()
        
        # Run the agent until it's finished
        while not self.agent.is_finished():
            # Track start time for this inference
            start_time = time.perf_counter()
            
            # Get response from OpenAI and process
            response = self.openai_client.chat.completions.create(
                model="gpt-4o",
                messages=self.agent.messages,
                tools=self.agent.get_tools(llm_provider=LLMProvider.OPEN_AI),
                tool_choice=self.agent.tool_choice,
                temperature=0.0
            )
            
            self.agent.add_messages([{"role": "assistant", "content": "the time now is : " + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}])
                       
            # Track token usage
            execution_tokens.worker.completion_tokens += response.usage.completion_tokens
            execution_tokens.worker.prompt_tokens += response.usage.prompt_tokens
            execution_tokens.worker.total_tokens += response.usage.total_tokens
            
            # Report LLM usage to Xpander
            self.agent.report_llm_usage(
                llm_response=response.model_dump(),
                llm_inference_duration=time.perf_counter() - start_time,
                llm_provider=LLMProvider.OPEN_AI
            )
                        
            self.agent.add_messages(response.model_dump())
            
            # Extract and execute any tool calls
            tool_calls = XpanderClient.extract_tool_calls(
                llm_response=response.model_dump(),
                llm_provider=LLMProvider.OPEN_AI
            )
            
            for tool_call in tool_calls:
                print("AI decided to call : ", tool_call.name)
                print("With the following payload : ", tool_call.payload)
            if tool_calls:
                print("Executing tool calls")
                responses = self.agent.run_tools(tool_calls=tool_calls)
                for response in responses:
                    print("Tool call function name : ", response.function_name)
                    print("HTTP Status code : ", response.status_code)
        
        # Process results
        execution_end_time = time.perf_counter()
        result = self.agent.retrieve_execution_result()
        thread_id = result.memory_thread_id
        print(f"Your thread ID is: {thread_id}")
        result_text = result.result
    
        # Report execution metrics to Xpander
        self.agent.report_execution_metrics(
            llm_tokens=execution_tokens,
            ai_model="gpt-4o"
        )
        
        print(f"Status: {result.status}")
        print(f"Result: {result_text}")
        print(f"Execution duration: {execution_end_time - execution_start_time:.2f} seconds")
    
        # Return both the result text and thread ID
        return result_text, thread_id

## 3. Initialize the Meeting Agent

Let's create an instance of our agent runner:

In [5]:
# Initialize the meeting agent
meeting_agent = MeetingAgent(
    openai_api_key=openai_api_key,
    xpander_api_key=xpander_api_key,
    agent_id=agent_id
)

meeting_agent.agent.instructions.general = f"""
You can access the user’s calendar but cannot schedule events.
Use the calendar to find meeting URLs if not provided.
To start a recording, you must have both the meeting URL and the meeting time in UTC.
This will return a recording bot ID.
You can only retrieve the transcript and video if you have this bot ID — no exceptions.
Do not execute any function unless it’s strictly required or explicitly requested by the user.
"""

meeting_agent.agent.instructions.role = """
You are meeting recorder AI agent with access to Google Calendar and Meeting Recording tools.
"""

meeting_agent.agent.instructions.goal = """
Your goal is to help users to record their meetings and get the assets of the recorded meeting using the recording bot id.
"""

print(f"Meeting agent initialized with ID: {meeting_agent.agent_id}")

Meeting agent initialized with ID: 6bb96105-3675-4d79-9dd3-8ae146cff007


## 4. Test the agent

Let's run the agent !

In [5]:
# Run the agent with a prompt to check calendar availability
prompt = """
Hi ! what can you do ? and what is the time now ?
"""

result, thread_id = meeting_agent.run(prompt=prompt)

# Save the thread ID for future conversations
user_thread_1 = thread_id

------------------------------------------------------------
Starting a new conversation thread
Processing Task: 


Hi ! what can you do ? and what is the time now ?



------------------------------------------------------------
AI decided to call :  CalendarListManagementGetUserCalendarListEntries
With the following payload :  {'bodyParams': {}, 'queryParams': {}, 'pathParams': {}, 'headers': {}}
Executing tool calls
Tool call function name :  CalendarListManagementGetUserCalendarListEntries
HTTP Status code :  200
AI decided to call :  CalendarEventManagementGetCalendarEventsById
With the following payload :  {'bodyParams': {}, 'queryParams': {'maxResults': 1, 'orderBy': 'startTime', 'singleEvents': True, 'timeMin': '2025-04-16T00:00:00Z', 'timeZone': 'Asia/Jerusalem'}, 'pathParams': {'calendarId': 'xpandersandbox@gmail.com'}, 'headers': {}}
Executing tool calls
Tool call function name :  CalendarEventManagementGetCalendarEventsById
HTTP Status code :  200
AI decided to call :  xpfi

## 5. Example 1: Retrieve Upcoming Calendar Events

Now that we've authorized access to Google Calendar, let's retrieve upcoming events:

In [6]:
# Use the same thread_id to maintain context from the authorization
prompt = """
Please list my upcoming meetings for the next 3 days. 
For each meeting, include the title, date, time, and participants if available.
"""

meeting_agent.run(prompt=prompt, thread_id=user_thread_1)

------------------------------------------------------------
Continuing conversation in thread: ce4d51c0-f6bd-4261-a535-cac50320fd0a
Processing Task: 


Please list my upcoming meetings for the next 3 days. 
For each meeting, include the title, date, time, and participants if available.



------------------------------------------------------------
AI decided to call :  CalendarEventManagementGetCalendarEventsById
With the following payload :  {'bodyParams': {}, 'queryParams': {'maxResults': 10, 'orderBy': 'startTime', 'singleEvents': True, 'timeMin': '2025-04-16T00:00:00Z', 'timeMax': '2025-04-19T00:00:00Z', 'timeZone': 'Asia/Jerusalem'}, 'pathParams': {'calendarId': 'xpandersandbox@gmail.com'}, 'headers': {}}
Executing tool calls
Tool call function name :  CalendarEventManagementGetCalendarEventsById
HTTP Status code :  200
AI decided to call :  xpfinish-agent-execution-finished
With the following payload :  {'bodyParams': {'result': 'Here are your upcoming meetings for the next 3 d

('Here are your upcoming meetings for the next 3 days:\n\n1. **Onboarding to xpander.ai**\n   - **Date & Time**: April 16, 2025, 13:00 - 14:00 (Asia/Jerusalem)\n   - **Participants**: Organizer - xpandersandbox@gmail.com\n   - **Link**: [Google Meet](https://meet.google.com/gxn-matc-dmh)\n\n2. **Daily Standup Meeting**\n   - **Date & Time**: April 17, 2025, 12:00 - 13:00 (UTC)\n   - **Participants**: team@xpander.ai\n   - **Link**: [Google Meet](https://meet.google.com/isn-nffb-tzs)\n\n3. **Customer Onboarding Session**\n   - **Date & Time**: April 17, 2025, 14:00 - 15:00 (UTC)\n   - **Participants**: customer@xpander.ai\n   - **Link**: [Google Meet](https://meet.google.com/fab-jwyd-bhn)\n\n4. **Team Retrospective**\n   - **Date & Time**: April 17, 2025, 16:00 - 17:00 (UTC)\n   - **Participants**: team@xpander.ai\n   - **Link**: [Google Meet](https://meet.google.com/pqk-vavr-qqh)\n\n5. **Product Demo with Client**\n   - **Date & Time**: April 18, 2025, 12:00 - 13:00 (UTC)\n   - **Parti

## 6. Example 2: Set Up Meeting Recording

Let's create a recording bot for an upcoming meeting:

In [7]:
# Create a meeting recorder for a specific meeting
prompt = """
Can you record all the meeting scheduled for today? 
"""

meeting_agent.run(prompt=prompt, thread_id=user_thread_1)

------------------------------------------------------------
Continuing conversation in thread: ce4d51c0-f6bd-4261-a535-cac50320fd0a
Processing Task: 


Can you record all the meeting scheduled for today? 



------------------------------------------------------------
AI decided to call :  XpanderMeetingManagementCreateRecordingBot
With the following payload :  {'bodyParams': {'meeting_url': 'https://meet.google.com/gxn-matc-dmh', 'scheduled_time': '2025-04-16T10:00:00Z'}, 'queryParams': {}, 'pathParams': {}, 'headers': {}}
Executing tool calls
Tool call function name :  XpanderMeetingManagementCreateRecordingBot
HTTP Status code :  200
AI decided to call :  XpanderMeetingManagementCheckRecorderStatus
With the following payload :  {'bodyParams': {}, 'queryParams': {}, 'pathParams': {'recorder_id': '61bcac95-5b0f-412f-9f4f-e3c1a954b636'}, 'headers': {}}
Executing tool calls
Tool call function name :  XpanderMeetingManagementCheckRecorderStatus
HTTP Status code :  200
AI decided to call

('The meeting scheduled for today, "Onboarding to xpander.ai," has been set up for recording. The recording bot is currently in the waiting room and will start recording once the meeting begins. The meeting is accessible via [this link](https://meet.google.com/gxn-matc-dmh).',
 'ce4d51c0-f6bd-4261-a535-cac50320fd0a')

## 7. Example 3: Check Recording Status

After setting up a recording, we can check its status:

In [8]:
# Check the status of our meeting recorder
# Note: In a real scenario, you'd use the recorder ID returned from the previous step
prompt = """
Check the status of the meeting recorder I just created for today's meeting
If it's not yet active, when will it start recording?
"""

result, thread_id = meeting_agent.run(prompt=prompt, thread_id=user_thread_1)

------------------------------------------------------------
Continuing conversation in thread: ce4d51c0-f6bd-4261-a535-cac50320fd0a
Processing Task: 


Check the status of the meeting recorder I just created for today's meeting
If it's not yet active, when will it start recording?



------------------------------------------------------------
AI decided to call :  XpanderMeetingManagementCheckRecorderStatus
With the following payload :  {'bodyParams': {}, 'queryParams': {}, 'pathParams': {'recorder_id': '61bcac95-5b0f-412f-9f4f-e3c1a954b636'}, 'headers': {}}
Executing tool calls
Tool call function name :  XpanderMeetingManagementCheckRecorderStatus
HTTP Status code :  200
AI decided to call :  xpfinish-agent-execution-finished
With the following payload :  {'bodyParams': {'result': "The meeting recorder for today's meeting has completed its task. Here are the assets:\n\n- **Video Recording**: [Download Video](https://links.xpander.ai/lrh3tcy)\n- **Transcript**: [Download Transcript](

In [6]:
user_thread_1 = "ce4d51c0-f6bd-4261-a535-cac50320fd0a"

## 8. Example 4: Send Recording Summary via Email

Once a recording is complete, we can request a summary to be sent via email:

In [7]:
# Request a summary of a completed meeting recording
prompt = """
Please send those assets to user@example.com with a beautiful email.
"""

result, thread_id = meeting_agent.run(prompt=prompt, thread_id=user_thread_1)

------------------------------------------------------------
Continuing conversation in thread: ce4d51c0-f6bd-4261-a535-cac50320fd0a
Processing Task: 


Please send those assets to user@example.com with a beautiful email.



------------------------------------------------------------
AI decided to call :  XpanderMessagingServiceSendEmailWithContent
With the following payload :  {'bodyParams': {'body_html': '<html><body><h2>Meeting Summary</h2><p>Dear Team,</p><p>Here is the summary of our recent meeting:</p><ul><li><strong>Main Discussion Points:</strong> The meeting focused on the onboarding process for new clients, improvements in our current workflow, and upcoming project deadlines.</li><li><strong>Action Items:</strong> Assign team members to specific tasks, follow up with clients for feedback, and prepare a report on the current project status by next week.</li></ul><p>You can access the meeting assets using the following links:</p><ul><li><a href="https://links.xpander.ai/lrh3tc

## Summary: Working with the Meeting Recorder Agent

In this notebook, we've explored how to use the AI Meeting Recorder Agent for various meeting-related tasks:

1. **Google Calendar Integration**:
   - Authorization and authentication
   - Retrieving upcoming meetings
   - Creating new calendar events

2. **Meeting Recording Capabilities**:
   - Setting up recording bots for meetings
   - Checking recording status
   - Processing completed recordings

3. **Email Notifications**:
   - Sending meeting summaries via email
   - Including action items and key points

4. **Memory and Context Preservation**:
   - Using thread IDs to maintain conversation context
   - Testing memory across multiple interactions


### Next Steps

To further enhance your use of the Meeting Recorder Agent:

1. Customize the agent with additional tools or capabilities
2. Integrate with other calendar systems or communication platforms
3. Develop more sophisticated workflows for specific meeting types
4. Implement authentication for multiple users

Visit the [Xpander documentation](https://docs.xpander.ai) for more information on extending your agent's capabilities.